In [55]:
import xml.etree.ElementTree as ET
from shapely.geometry import LineString
import pandas as pd

In [57]:
def generate_maze(maze_layout, start_positions=None, goals=None):
    
    num_rows = len(maze_layout)
    num_cols = len(maze_layout[0]) if num_rows > 0 else 0

    added_walls = []

    def add_wall(x1, y1, x2, y2):
        wall = tuple([(x1, y1), (x2, y2)])
        if wall not in added_walls:
            added_walls.append(wall)

    # Generate internal walls based on the maze layout
    for i, row in enumerate(maze_layout):
        for j, cell in enumerate(row):
            cell_x = j - num_cols / 2 + 0.5  # Center x position of the cell
            cell_y = num_rows / 2 - i - 0.5  # Center y position of the cell

            if cell[0] == 1:  # North wall
                if i == 0:  # Top row cells
                    add_wall(cell_x-0.5, cell_y+0.5, cell_x+0.5, cell_y+0.5)
                elif maze_layout[i-1][j][2] == 0:  # If no South wall in the cell above, add North wall
                    add_wall(cell_x-0.5, cell_y+0.5, cell_x+0.5, cell_y+0.5)

            if cell[1] == 1:  # East wall
                add_wall(cell_x+0.5, cell_y+0.5, cell_x+0.5, cell_y-0.5)

            if cell[2] == 1:  # South wall
                add_wall(cell_x-0.5, cell_y-0.5, cell_x+0.5, cell_y-0.5)

            if cell[3] == 1:  # West wall
                if j == 0:  # Leftmost column cells
                    add_wall(cell_x-0.5, cell_y+0.5, cell_x-0.5, cell_y-0.5)
                elif maze_layout[i][j-1][1] == 0:  # If no East wall in the cell to the left, add West wall
                    add_wall(cell_x-0.5, cell_y+0.5, cell_x-0.5, cell_y-0.5)

    return added_walls


In [63]:
def are_parallel(line1, line2):
    """Check if two lines are parallel by comparing their direction vectors."""
    x1, y1 = line1.coords[0]
    x2, y2 = line1.coords[-1]
    x3, y3 = line2.coords[0]
    x4, y4 = line2.coords[-1]

    # Calculate direction vectors
    vector1 = (x2 - x1, y2 - y1)
    vector2 = (x4 - x3, y4 - y3)

    # Calculate cross product (should be zero if vectors are parallel)
    cross_product = vector1[0] * vector2[1] - vector1[1] * vector2[0]

    return cross_product == 0

def are_touching(line1, line2):
    """Check if two lines are touching (end of one is start of another)."""
    return line1.coords[-1] == line2.coords[0]

def combine_walls_if_parallel_and_touching(walls):
    combined_walls = []
    while walls:
        current_wall = walls.pop(0)
        i = 0
        while i < len(walls):
            if are_parallel(current_wall, walls[i]) and are_touching(current_wall, walls[i]):
                current_wall = LineString(list(current_wall.coords) + list(walls[i].coords[1:]))
                walls.pop(i)
            else:
                i += 1
        combined_walls.append(current_wall)
    return combined_walls

def walls_and_positions_to_xml(combined_walls, start_positions=None, goals=None):
    root = ET.Element("world")
    for wall in combined_walls:
        (x1, y1), (x2, y2) = wall.coords[0], wall.coords[-1]
        ET.SubElement(root, "wall", x1=str(x1), y1=str(y1), x2=str(x2), y2=str(y2))

    if start_positions:
        start_positions_element = ET.SubElement(root, "startPositions")
        for pos in start_positions:
            ET.SubElement(start_positions_element, "pos", x=str(pos["x"]), y=str(pos["y"]), theta=str(pos["theta"]))

    if goals:
        for goal in goals:
            ET.SubElement(root, "goal", id=str(goal["id"]), x=str(goal["x"]), y=str(goal["y"]))

    return ET.tostring(root, encoding='unicode', method='xml')

In [60]:
def save_maze_to_file(xml_string, file_name):
    # Ensure the file name has an .xml extension
    if not file_name.endswith('.xml'):
        file_name += '.xml'
    
    # Write the XML string to the file
    with open(file_name, 'w', encoding='utf-8') as file:
        file.write(xml_string)

    print(f"Maze saved to {file_name}")

In [77]:
maze_layout = [
    [[1, 0, 0, 1], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 1, 0, 0]],  # First row with North walls
    [[0, 1, 0, 1], [1, 0, 1, 1], [1, 0, 1, 0], [1, 0, 1, 0], [0, 1, 1, 0]],  # Second row with South and West walls
    [[0, 0, 1, 1], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 1, 0, 0]],  # Third row
    [[1, 0, 0, 1], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 1, 0, 0]],  # Fourth row
    [[0, 0, 1, 1], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 1, 1, 0]]   # Fifth row
]

# Generate the walls
added_walls = generate_maze(maze_layout)

# Convert to LineStrings
line_segments = [LineString([wall[0], wall[1]]) for wall in added_walls]

# Combine parallel and touching walls
combined_walls = combine_walls_if_parallel_and_touching(line_segments)

# Example start and goal positions
start_positions = [{"x": 2.0, "y": -2.0, "theta": 3.14}]
goals = [{"id": 1, "x": -1.0, "y": 1.0}]

# Convert to XML
xml_output = walls_and_positions_to_xml(combined_walls, start_positions=start_positions, goals=goals)

# Output the XML
print(xml_output)

save_maze_to_file(xml_output,'../../WebotsSim/worlds/Fall24/maze1')

<world><wall x1="-2.5" y1="2.5" x2="2.5" y2="2.5" /><wall x1="-2.5" y1="2.5" x2="-2.5" y2="-2.5" /><wall x1="-1.5" y1="1.5" x2="1.5" y2="1.5" /><wall x1="2.5" y1="2.5" x2="2.5" y2="-2.5" /><wall x1="-1.5" y1="1.5" x2="-1.5" y2="0.5" /><wall x1="-1.5" y1="0.5" x2="2.5" y2="0.5" /><wall x1="-2.5" y1="-0.5" x2="2.5" y2="-0.5" /><wall x1="-1.5" y1="-1.5" x2="2.5" y2="-1.5" /><wall x1="-2.5" y1="-2.5" x2="2.5" y2="-2.5" /><startPositions><pos x="2.0" y="-2.0" theta="3.14" /></startPositions><goal id="1" x="-1.0" y="1.0" /></world>
Maze saved to ../../WebotsSim/worlds/Fall24/maze1.xml


In [71]:
maze_layout = [
    [[1, 0, 0, 1], [1, 1, 0, 0], [1, 0, 0, 1]],
    [[0, 1, 1, 1], [1, 1, 1, 1], [0, 1, 1, 1]],
    [[0, 0, 1, 1], [0, 1, 1, 0], [0, 0, 1, 1]]
]


# Generate the walls
added_walls = generate_maze(maze_layout)

# Convert to LineStrings
line_segments = [LineString([wall[0], wall[1]]) for wall in added_walls]

# Combine parallel and touching walls
combined_walls = combine_walls_if_parallel_and_touching(line_segments)

# Example start and goal positions
start_positions = [{"x": -1.5, "y": 1.5, "theta": 3.14}]
goals = [{"id": 1, "x": 2.5, "y": -2.5}]

# Convert to XML
xml_output = walls_and_positions_to_xml(combined_walls, start_positions=start_positions, goals=goals)

# Output the XML
print(xml_output)

save_maze_to_file(xml_output,'../../WebotsSim/worlds/mazes/samples/test1')

<world><wall x1="-1.5" y1="1.5" x2="1.5" y2="1.5" /><wall x1="-1.5" y1="1.5" x2="-1.5" y2="-1.5" /><wall x1="0.5" y1="1.5" x2="0.5" y2="-1.5" /><wall x1="-0.5" y1="0.5" x2="-0.5" y2="-0.5" /><wall x1="-1.5" y1="-0.5" x2="1.5" y2="-0.5" /><wall x1="-0.5" y1="0.5" x2="0.5" y2="0.5" /><wall x1="1.5" y1="0.5" x2="1.5" y2="-0.5" /><wall x1="-1.5" y1="-1.5" x2="1.5" y2="-1.5" /><startPositions><pos x="-1.5" y="1.5" theta="3.14" /></startPositions><goal id="1" x="2.5" y="-2.5" /></world>
Maze saved to ../../WebotsSim/worlds/mazes/samples/test1.xml


In [72]:
maze_layout = [
    [[1, 1, 0, 1], [1, 0, 0, 1], [1, 0, 0, 1], [1, 1, 0, 0]],
    [[0, 1, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0], [0, 1, 0, 0]],
    [[0, 1, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0], [0, 1, 0, 0]],
    [[0, 1, 1, 1], [0, 0, 1, 0], [0, 0, 1, 0], [0, 1, 1, 0]]
]


# Generate the walls
added_walls = generate_maze(maze_layout)

# Convert to LineStrings
line_segments = [LineString([wall[0], wall[1]]) for wall in added_walls]

# Combine parallel and touching walls
combined_walls = combine_walls_if_parallel_and_touching(line_segments)

# Example start and goal positions
start_positions = [{"x": -1.5, "y": 1.5, "theta": 3.14}]
goals = [{"id": 1, "x": 2.5, "y": -2.5}]

# Convert to XML
xml_output = walls_and_positions_to_xml(combined_walls, start_positions=start_positions, goals=goals)

# Output the XML
print(xml_output)

save_maze_to_file(xml_output,'../../WebotsSim/worlds/mazes/samples/test2')

<world><wall x1="-2.0" y1="2.0" x2="2.0" y2="2.0" /><wall x1="-1.0" y1="2.0" x2="-1.0" y2="-2.0" /><wall x1="-2.0" y1="2.0" x2="-2.0" y2="-2.0" /><wall x1="0.0" y1="2.0" x2="0.0" y2="1.0" /><wall x1="2.0" y1="2.0" x2="2.0" y2="-2.0" /><wall x1="-2.0" y1="-2.0" x2="2.0" y2="-2.0" /><startPositions><pos x="-1.5" y="1.5" theta="3.14" /></startPositions><goal id="1" x="2.5" y="-2.5" /></world>
Maze saved to ../../WebotsSim/worlds/mazes/samples/test2.xml


In [73]:
maze_layout = [
    [[1, 0, 0, 1], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 1, 0, 0]],
    [[0, 0, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 1, 0, 0]],
    [[0, 0, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 1, 0, 0]],
    [[0, 0, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 1, 0, 0]],
    [[0, 0, 1, 1], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 1, 1, 0]]
]


# Generate the walls
added_walls = generate_maze(maze_layout)

# Convert to LineStrings
line_segments = [LineString([wall[0], wall[1]]) for wall in added_walls]

# Combine parallel and touching walls
combined_walls = combine_walls_if_parallel_and_touching(line_segments)

# Example start and goal positions
start_positions = [{"x": -1.5, "y": 1.5, "theta": 3.14}]
goals = [{"id": 1, "x": 2.5, "y": -2.5}]

# Convert to XML
xml_output = walls_and_positions_to_xml(combined_walls, start_positions=start_positions, goals=goals)

# Output the XML
print(xml_output)

save_maze_to_file(xml_output,'../../WebotsSim/worlds/mazes/samples/test3')

<world><wall x1="-2.5" y1="2.5" x2="2.5" y2="2.5" /><wall x1="-2.5" y1="2.5" x2="-2.5" y2="-2.5" /><wall x1="2.5" y1="2.5" x2="2.5" y2="-2.5" /><wall x1="-2.5" y1="-2.5" x2="2.5" y2="-2.5" /><startPositions><pos x="-1.5" y="1.5" theta="3.14" /></startPositions><goal id="1" x="2.5" y="-2.5" /></world>
Maze saved to ../../WebotsSim/worlds/mazes/samples/test3.xml


In [74]:
maze_layout = [
    [[1, 0, 0, 1], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 1, 0, 0]],
    [[0, 1, 1, 1], [0, 1, 1, 1], [0, 1, 1, 1], [0, 1, 1, 1], [0, 1, 1, 1]],
    [[0, 0, 1, 1], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 1, 1, 0]]
]


# Generate the walls
added_walls = generate_maze(maze_layout)

# Convert to LineStrings
line_segments = [LineString([wall[0], wall[1]]) for wall in added_walls]

# Combine parallel and touching walls
combined_walls = combine_walls_if_parallel_and_touching(line_segments)

# Example start and goal positions
start_positions = [{"x": -1.5, "y": 1.5, "theta": 3.14}]
goals = [{"id": 1, "x": 2.5, "y": -2.5}]

# Convert to XML
xml_output = walls_and_positions_to_xml(combined_walls, start_positions=start_positions, goals=goals)

# Output the XML
print(xml_output)

save_maze_to_file(xml_output,'../../WebotsSim/worlds/mazes/samples/test4')

<world><wall x1="-2.5" y1="1.5" x2="2.5" y2="1.5" /><wall x1="-2.5" y1="1.5" x2="-2.5" y2="-1.5" /><wall x1="2.5" y1="1.5" x2="2.5" y2="-1.5" /><wall x1="-1.5" y1="0.5" x2="-1.5" y2="-0.5" /><wall x1="-2.5" y1="-0.5" x2="2.5" y2="-0.5" /><wall x1="-0.5" y1="0.5" x2="-0.5" y2="-0.5" /><wall x1="0.5" y1="0.5" x2="0.5" y2="-0.5" /><wall x1="1.5" y1="0.5" x2="1.5" y2="-0.5" /><wall x1="-2.5" y1="-1.5" x2="2.5" y2="-1.5" /><startPositions><pos x="-1.5" y="1.5" theta="3.14" /></startPositions><goal id="1" x="2.5" y="-2.5" /></world>
Maze saved to ../../WebotsSim/worlds/mazes/samples/test4.xml


In [75]:
maze_layout = [
    [[1, 1, 0, 1], [1, 0, 0, 1], [1, 1, 0, 0]],
    [[0, 1, 0, 1], [0, 1, 0, 0], [0, 1, 0, 0]],
    [[0, 1, 0, 1], [0, 0, 0, 0], [0, 1, 0, 0]],
    [[0, 1, 0, 1], [0, 0, 0, 0], [0, 1, 0, 0]],
    [[0, 1, 1, 1], [0, 0, 1, 0], [0, 1, 1, 0]]
]


# Generate the walls
added_walls = generate_maze(maze_layout)

# Convert to LineStrings
line_segments = [LineString([wall[0], wall[1]]) for wall in added_walls]

# Combine parallel and touching walls
combined_walls = combine_walls_if_parallel_and_touching(line_segments)

# Example start and goal positions
start_positions = [{"x": -1.5, "y": 1.5, "theta": 3.14}]
goals = [{"id": 1, "x": 2.5, "y": -2.5}]

# Convert to XML
xml_output = walls_and_positions_to_xml(combined_walls, start_positions=start_positions, goals=goals)

# Output the XML
print(xml_output)

save_maze_to_file(xml_output,'../../WebotsSim/worlds/mazes/samples/test5')

<world><wall x1="-1.5" y1="2.5" x2="1.5" y2="2.5" /><wall x1="-0.5" y1="2.5" x2="-0.5" y2="-2.5" /><wall x1="-1.5" y1="2.5" x2="-1.5" y2="-2.5" /><wall x1="1.5" y1="2.5" x2="1.5" y2="-2.5" /><wall x1="0.5" y1="1.5" x2="0.5" y2="0.5" /><wall x1="-1.5" y1="-2.5" x2="1.5" y2="-2.5" /><startPositions><pos x="-1.5" y="1.5" theta="3.14" /></startPositions><goal id="1" x="2.5" y="-2.5" /></world>
Maze saved to ../../WebotsSim/worlds/mazes/samples/test5.xml
